Imports

In [20]:
import torch
import re
import numpy as np
import subprocess
import os
from scipy.optimize import least_squares

# Get K1 and K2 (Checkerboard)

opens camera and performs camera calibration using the checkerboard.

In [21]:
# # Path to the Python script
# script_path = 'Camera-Calibration/calibration_GUI.py'

# def calibrate(camera_num): 
#     """
#     Calibrates camera(camera_num) by running the calibration_GUI script.
    
#     :param camera_num: the number of the camera. (needs to be a string)
#     :return: intrinsic matrix K
#     """
#     # Run the script and capture the output
#     output = subprocess.check_output(['python', script_path, '--cam_num', camera_num])

#     # Decode the output and process it to extract the matrix A
#     output_str = output.decode('utf-8').strip()

#     cleaned_output = output_str.replace('[', '').replace(']', '').replace(',', '').strip()

#     return cleaned_output

# # since we use a+, if we don't delete the file manually, it will write again and again.
# if os.path.exists("./output/intrinsic.txt"): 
# 	os.remove("./output/intrinsic.txt")
	
# if os.path.exists("./output/extrinsic.txt"):
# 	os.remove("./output/extrinsic.txt")

# A = np.array([list(map(float, row.split())) for row in calibrate("0").splitlines()])
# K1 = A
# A = np.array([list(map(float, row.split())) for row in calibrate("2").splitlines()])
# K2 = A

# K1[0][1] = 0 # set skew(distortion to 0)
# K2[0][1] = 0
# # Use A as needed
# print("Intrinsic Matrix K1:", K1,"\nIntrinsic Matrix K2:", K2)


# Extract K1 and K2 from txt file

for convenience, extract the intrinsics from the txt files.

In [22]:
with open('output/intrinsic.txt', 'r') as f:
    K = f.read()
numbers1 = re.findall(r"[-+]?\d*\.\d+|\d+", K)

list_K = [float(num) for num in numbers1]
K1 = np.array(list_K[:9]).reshape(3,3)
K2 = np.array(list_K[9:]).reshape(3,3)

# Set skew(distortion to 0)
K1[0][1] = 0 
K2[0][1] = 0

# Print out the Ks
print(f"K1: \n{K1}")
print(f"K2: \n{K2}")

K1: 
[[294.69022161   0.         432.79487108]
 [  0.         282.0225284  240.17817173]
 [  0.           0.           1.        ]]
K2: 
[[314.16943092   0.         383.56227906]
 [  0.         299.69405192 266.91023375]
 [  0.           0.           1.        ]]


Extract the fx, fy, u0, v0 (focal lengths and principal point from both K)

In [23]:
K1_params = [K1[0, 0], K1[1, 1], K1[0, 2], K1[1, 2]]

print("K1_params:", K1_params)

K2_params = [K2[0, 0], K2[1, 1], K2[0, 2], K2[1, 2]]

print("K2_params:", K2_params)

K1_params: [np.float64(294.69022161), np.float64(282.0225284), np.float64(432.79487108), np.float64(240.17817173)]
K2_params: [np.float64(314.16943092), np.float64(299.69405192), np.float64(383.56227906), np.float64(266.91023375)]


# Extract matches from txt files

txt file containing the matches is called `cam_match.txt`

In [24]:
import os
print(os.getcwd())


c:\Users\70929\PoseGuidedCalibration\PoseGuidedCalibration


In [25]:
with open('data/cam_match.txt', 'r') as file:
    content = file.read()

In [33]:
with open('data/cam_match.txt', 'r') as file:
    content = file.read()
def extract_homonymous_pairs(content, threshold):
    """
    Extracts keypoint pairs with confidence higher than a given threshold.
    
    :param list_keypoints: List of keypoints in format [x, y], [x1, y1], [confidence]
    :param threshold: Confidence threshold
    :return: Filtered coordinates as a numpy array
    """
    lines = content.strip().splitlines()
    pattern = re.compile(r'\[(\d+), (\d+)\], \[(\d+), (\d+)\], \[(\d+\.\d+)\]')
    matches_final = []

    for i, line in enumerate(lines):
        bruh = []
        matches = pattern.findall(line)

        for j, match in enumerate(matches):
            keypoint1 = [int(match[0]), int(match[1])]
            x0 = keypoint1[0]
            y0 = keypoint1[1]
            keypoint2 = [int(match[2]), int(match[3])]
            x1 = keypoint2[0]
            y1 = keypoint2[1]
            confidence = float(match[4])

            if confidence > threshold:
                
                bruh.append([x0, y0, x1, y1])
                print(f"Keypoint 1({j}): {keypoint1}, Keypoint 2({j}): {keypoint2}, Confidence: {confidence}")
        # print(f"{i} : {bruh}")
        if bruh:
            matches_final.append(bruh)
            print(f"(image {i})\n")

        
    return matches_final

print("Homonymous Points:\n")
matches_final = extract_homonymous_pairs(content, 0.4)
print(matches_final)


Homonymous Points:

[]


# Fundamental & Essential Matrix Functions

below are the functions used for calculating the fundamental and essential matrices

In [29]:
def compute_fundamental_matrix(correspondences):
    """
    Compute the fundamental matrix F using the 8-point algorithm.

    :params: correspondences: Nx4 array of corresponding points [u1, v1, u2, v2]
    :return: fundamental matrix "F"
    """
    A = []
    for u1, v1, u2, v2 in correspondences: # [1 X 9] matrix
        A.append([u1*u2, u1*v2, u1, v1*u2, v1*v2, v1, u2, v2, 1]) # there's a '1' because of the homogeneous equation
    A = np.array(A)

    # Af = 0 using SVD
    _, _, V = np.linalg.svd(A)

    # print(f"initial V = {V}")
    F = V[-1].reshape(3, 3) #found in the right singular vector corresponding to the smallest singular value

    # rank 2 constraint
    U, S, V = np.linalg.svd(F)

    # print(f"s = {S}\n")

    S[2] = 0
    F = U @ np.diag(S) @ V
    
    return F / F[2,2]  # Normalize

# for i, match in enumerate(matches_final):
#     print(match)
#     F = compute_fundamental_matrix(match)
#     print(f"F{i} = {F}")


In [30]:
def compute_essential_matrix(F, K1, K2):
    """
    Compute the essential matrix E from F and camera intrinsics.
    F: 3x3 fundamental matrix
    K1, K2: 3x3 intrinsic camera matrices
    """
    E = K2.T @ F @ K1
    return E

Main loop for calculating Fundamental and Essential matrices

In [31]:
# Prompt user for image number
# choice = int(input("Image number? (start from 0)"))
choice = 8

# Calculate the fundamental matrix with the 8-point algorithm
match = matches_final[choice]
print(f"matches for image {choice}:\n{match}\n")

F = compute_fundamental_matrix(match)
print(f"F = \n{F}")

# Calculate the essential matrix
E = compute_essential_matrix(F, K1, K2)
print(f"\nE = \n{E}")

IndexError: list index out of range

Handling the coordinates and splitting them to their respective lists
&
Convert it into homogeneous form

In [ ]:
points2d_1 = []  # 2D points from first camera
points2d_2 = []  # Corresponding 2D points from second camera
for x1,y1,x2,y2 in match:
    points2d_1.append([x1,y1])
    points2d_2.append([x2,y2])

# Convert it into a numpy array
X1 = np.array(points2d_1)
X2 = np.array(points2d_2)

# convert it into a homogeneous point
X1_homogeneous = np.hstack([X1, np.ones((X1.shape[0], 1))])
X2_homogeneous = np.hstack([X2, np.ones((X2.shape[0], 1))])
print(f"X1 (homogeneous): \n {X1_homogeneous}\n")
print(f"X2 (homogeneous): \n {X2_homogeneous}") 

# Decompose Essential Matrix

With 4 combinations of R and t, choose the best one.

In [ ]:
def decompose_essential_matrix(E):
    """
    Perform Singular Value Decomposition (SVD) of the essential matrix.
    
    :params: E: Essential Matrix.
    :return: X: Array of 3D points in homogeneous coordinates, shape (N, 4).
    """
    U, _, Vt = np.linalg.svd(E)

    # Ensure we have a proper rotation matrix
    if np.linalg.det(U) < 0:
        U[:, -1] *= -1
    if np.linalg.det(Vt) < 0:
        Vt[-1, :] *= -1

    # Possible rotation matrices from the decomposition
    W = np.array([[0, -1, 0], 
                  [1,  0, 0], 
                  [0,  0, 1]])

    # Two possible rotations
    R1 = U @ W @ Vt
    R2 = U @ W.T @ Vt

    # Two possible translations (up to scale, so we use the third column of U)
    t1 = U[:, 2]
    t2 = -U[:, 2]

    return R1, R2, t1, t2

# Decompose the essential matrix
R1, R2, t1, t2 = decompose_essential_matrix(E)

# Output the four possible solutions
print("Possible solutions for R and t:")
print("R1:\n", R1)
print("t1:\n", t1)
print("R2:\n", R2)
print("t2:\n", t2)

# Triangulate Point

Get the 3D point(x, y, z, 1) in homogeneous coordinates by triangulating the 2D points.

In [12]:
def triangulate_points(x1, x2, P1, P2):
    """Triangulates 3D points from corresponding 2D points in two images.
    
    Args:
    x1: Array of 2D points in the first image (homogeneous coordinates), shape (N, 3).
    x2: Array of 2D points in the second image (homogeneous coordinates), shape (N, 3).
    P1: Projection matrix of the first camera, shape (3, 4).
    P2: Projection matrix of the second camera, shape (3, 4).
    
    Returns:
    X: Array of 3D points in homogeneous coordinates, shape (N, 4).
    """
    assert x1.shape == x2.shape, "x1 and x2 must have the same shape."
    
    # Initialize list to store the 3D points
    X = []

    # Iterate over pairs of points
    for i in range(x1.shape[0]):
        # Construct matrix A for homogeneous equation system
        A = np.zeros((4, 4))
        A[0] = x1[i, 0] * P1[2] - P1[0]
        A[1] = x1[i, 1] * P1[2] - P1[1]
        A[2] = x2[i, 0] * P2[2] - P2[0]
        A[3] = x2[i, 1] * P2[2] - P2[1]

        # Solve the equation AX = 0 using SVD
        U, S, Vt = np.linalg.svd(A)
        X_nonhomogeneous = Vt[-1]
        X_homogeneous = X_nonhomogeneous / X_nonhomogeneous[-1]  # Convert to homogeneous coordinates
        # Append the result to the list
        X.append(X_homogeneous)

    # Convert list to a numpy array
    X = np.array(X)
    
    return X

# Find the correct solution

Since decomposition of E will have four different solutions[(R1, t1), (R1, -t1), (R2, t1), (R2, -t1)]

we need to find the one that is "in front of" the camera in which `depth > 0` for both camera

In [13]:
import numpy as np

def cheirality_check(X, P1, P2):
    """Performs a cheirality check to determine if 3D points are in front of both cameras.
    
    Args:
    X: Array of triangulated 3D points in homogeneous coordinates, shape (N, 4).
    P1: Projection matrix of the first camera, shape (3, 4).
    P2: Projection matrix of the second camera, shape (3, 4).
    
    Returns:
    Count of points that are in front of both cameras.
    """
    # Compute the depth of the points in both cameras
    depth1 = (P1[2, :] @ X.T)  # Z coordinate in camera 1 coordinate system
    depth2 = (P2[2, :] @ X.T)  # Z coordinate in camera 2 coordinate system

    print(f"depth for camera 1:\n{depth1}\n")
    print(f"depth for camera 2:\n{depth2}")
    # The points are in front of both cameras if both depths are positive
    count = np.sum((depth1 > 0) & (depth2 > 0))  # Count points in front of both cameras
    print("\nNumber of points in front of the camera:",count)
    return count

def find_correct_solution(R1, R2, t1, t2, x1, x2, K1, K2):
    """
    Finds the correct (R, t) combination by checking which solution keeps most points in front of both cameras.
    
    Args:
    R1, R2: Two possible rotation matrices.
    t1, t2: Two possible translation vectors.
    x1: Array of 2D points in the first image (homogeneous coordinates), shape (N, 3).
    x2: Array of 2D points in the second image (homogeneous coordinates), shape (N, 3).
    K1: Intrinsic matrix of the first camera, shape (3, 3).
    K2: Intrinsic matrix of the second camera, shape (3, 3).
    
    Returns:
    The correct (R, t) combination.
    """
    # Projection matrix for the first camera
    P1 = K1 @ np.hstack((np.eye(3), np.zeros((3, 1))))  # [I | 0]
    
    # Four combinations of R and t
    combinations = [
        (R1, t1),
        (R1, t2),
        (R2, t1),
        (R2, t2)
    ]

    best_count = 0

    # Check all four solutions
    for R, t in combinations:
        print("-------------------------------------------------------")
        # Second camera projection matrix [R | t]
        P2 = K2 @ np.hstack((R, t.reshape(-1, 1)))
        
        # Triangulate the 3D points for the current combination
        X = triangulate_points(x1, x2, P1, P2)  # Triangulate multiple 3D points
        print(f"R: \n{R}")
        print(f"\nt:\n{t}\n")
        print(f"triangulation:\n{X}\n")
        # Perform cheirality check
        
        count = cheirality_check(X, P1, P2)  # Check how many points are in front of both cameras
        
        # Update the best solution if more points are in front of both cameras
        if count > best_count:
            best_count = count
            best_R = R
            best_t = t
            best_triangulation = X
    
    return best_R, best_t, best_triangulation


Main loop for finding best R and t

In [ ]:

best_R, best_t, best_triangulation = find_correct_solution(R1, R2, t1, t2, X1_homogeneous, X2_homogeneous, K1, K2)
print(f"-------------------------------------------------------------------")
print(f"-------------------------------------------------------------------")
print(f"\nbest_R: \n{best_R} \n")
print(f"best_t: \n{best_t} \n")
print(f"best_triangulation: \n{best_triangulation}")

Setting up the R and t for both cameras

In [ ]:
R_1 = np.eye(3)
t_1 = np.zeros(3)

print(f"R_1:\n{R_1}\n")
print(f"t_1:\n{t_1}\n")

# R and t are flattened as a 1D array for optimization
R_t_1 = np.hstack([R_1.flatten(), t_1])  # Flatten R (rotation matrix) and append t (translation vector)

print("R_t_1:\n", R_t_1)


In [ ]:
R_2, t_2 = best_R,best_t

print(f"R_2:\n{R_2}\n")
print(f"t_2:\n{t_2}\n")

# R and t are flattened as a 1D array for optimization
R_t_2 = np.hstack([R_2.flatten(), t_2]) # Flatten R (rotation matrix) and append t (translation vector)

print("R_t_2:\n", R_t_2)

# Reprojection Loss

After triangulating the points, we need to reproject it back to 2D as well as calculate the loss (between the image detected points and the reprojected points).

Functions for constructing K and [R | t]

In [17]:
def construct_K_matrix(params):
    fx, fy, u0, v0 = params
    return np.array([
        [fx, 0, u0],
        [0, fy, v0],
        [0,  0,  1]
    ])

def extract_R_t(R_t):
    R = R_t[:9].reshape(3,3)
    t = R_t[9:]
    return R, t

Project points & normalization function

In [18]:
def project_points(params, R_t, X_3d):
    """
    Project 3D points to 2D using the camera parameters (not normalized)

    :param params: Parameters of K [fx, fy, u0, v0]
    :param R_t: 1D Flattened rotation and transalation matrix
    :param X_3d: 3D Triangulated points [x, y, z, 1]
    :return: 2D Projected points [x, y, z]

    """
    K = construct_K_matrix(params)

    R, t = extract_R_t(R_t)

    projected_points = []
    P = K @ np.hstack((R, t.reshape(-1, 1)))

    for i in X_3d:
        x = np.dot(P, i)
        projected_points.append(x)
    
    projected_points = np.array(projected_points)

    return projected_points

def normalize_points(points):
    """
    Normalizes given points.

    :param points: 2D Projected points [x, y, z]
    :return: Normalized 2D Projected points[x/z, y/z, 1]

    """
    points_normalized = points / points[:, 2][:, np.newaxis]
    
    # Remove the third dimension
    points_normalized_2dim = np.delete(points_normalized, 2, axis=1)

    return points_normalized, points_normalized_2dim


Loss function

In [19]:
def loss_function(params, R_t, X_3d, x_2d):

    x_proj = project_points(params, R_t, X_3d) # projected coordinates (2D)
    x_proj = np.delete(x_proj, 2, axis=1) # removing the third element

    error = [0]
    x_2d = np.array(x_2d)
    for n, point in enumerate(x_2d):
        error_now = np.abs((point * X_3d[n, 2]) - x_proj[n])
        # print((point * X[n, 2]), "-", x_proj[n])
        # print(f"= {error_now}")
        error_now_sum = np.sum(error_now)
        error += error_now_sum

    # print("error before division", error, "/", (n+1))
    error /= (n+1)
    print(f"error after division: {error}")

    return error

Main Loop for Reprojecting Error (for displaying purposes).

In [ ]:
projected_points= project_points(K1_params, R_t_1, best_triangulation)

print(f"Projected_points:\n{projected_points}\n")

# Remove the third element
projected_points_2dim = np.delete(projected_points, 2, axis=1)

print(f"Projected_points_2dim:\n{projected_points_2dim}\n")

normalized_projected_points, normalized_projected_points_2dim = normalize_points(projected_points)

print(f"normalized_projected_points:\n{normalized_projected_points}\n")
print(f"normalized_projected_points_2dim:\n{normalized_projected_points_2dim}\n")

print("\n--------------------------------------------------")

# Calculate the losses
loss_1 = loss_function(K1_params, R_t_1, best_triangulation, points2d_1)
loss_2 = loss_function(K2_params, R_t_2, best_triangulation, points2d_2)

# Print the losses
print(f"\nloss_1: {loss_1}")
print(f"loss_2: {loss_2}")

# Plotting

plotting functions are important! Only print the normalized 2 dimension points.

In [21]:
import matplotlib.pyplot as plt

def plot_points(points, name='Scatter Plot of New Coordinates'):
    x_coords, y_coords = zip(*points)
    width, height = 640, 480 

    plt.figure(figsize=(6,5))
    plt.scatter(x_coords, y_coords, c='red', marker='x')
    plt.title(name)
    plt.xlabel('X Coordinates')
    plt.ylabel('Y Coordinates')
    plt.xlim(0, width)
    plt.ylim(height, 0)
    plt.grid(True)
    plt.show()


Again, just for displaying.

In [ ]:
plot_points(normalized_projected_points_2dim)

# Optimization of Intrinsic Parameters & Joint Optimization

In [23]:
def optimize_intrinsic_parameters(params, R_t, X_3d, x_2d):
    """
    Optimize intrinsic parameters using least squares

    :param params: Parameters of K [fx, fy, u0, v0].
    :param R_t: 1D Flattened rotation and transalation matrix.
    :param X_3d: 3D Triangulated points [x, y, z, 1].
    :param  x_2d: Image detected joints (groundtruth).
    :return:  Optimized K, intrinsic parameter

    """

    result = least_squares(loss_function, params, args=(R_t, X_3d, x_2d))
    
    optimized_params = result.x
    
    return optimized_params

Main Loop for intrinsic optimization

In [ ]:
# Optimize for camera 1
K1_optimized_params = optimize_intrinsic_parameters(K1_params, R_t_1, best_triangulation, points2d_1)

print("\n-----------------------------------------------------------------------------")
# Optimize for camera 2
K2_optimized_params = optimize_intrinsic_parameters(K2_params, R_t_2, best_triangulation, points2d_2)

print("\n-----------------------------------------------------------------------------\n")

print("Optimized K1:")
print(K1_optimized_params)

print("\nOptimized K2:")
print(K2_optimized_params)

Debating whether to run this or not

In [ ]:
K1_new = construct_K_matrix(K1_optimized_params)
K2_new = construct_K_matrix(K2_optimized_params)

P1 = K1_new @ np.hstack((np.eye(3), np.zeros((3, 1))))  # [I | 0]
P2 = K2_new @ np.hstack((best_R, best_t.reshape(-1, 1)))

X_3D_nigga = triangulate_points(X1_homogeneous, X2_homogeneous, P1, P2)

x_2d = project_points(K1_optimized_params, R_t_1, X_3D_nigga)

print("x_2d:",x_2d)

normalized, normalized_2d = normalize_points(x_2d)

print("\nnormalized\n", normalized, "\normalized_2d\n", normalized_2d)

Plotting the results

In [ ]:
plot_points(normalized_projected_points_2dim, "before optimized K1 and K2"), plot_points(normalized_2d, "with optimized K1 and K2"), plot_points(points2d_1, "ground truth")

In [27]:
def optimize_extrinsic_parameters(params, R_t, X_3d, x_2d):
    """
    Optimize extrinsic parameters using least squares

    params: K1_init, K2_init: Initial 3x3 intrinsic matrices for camera 1 and 2
    params: R, t: Relative extrinsic parameters between the two cameras
    params: X_3d: Nx3 array of 3D points (triangulated human body joints)
    params: x_2d_1, x_2d_2: Nx2 arrays of 2D points for camera 1 and 2

    """
    def loss_function_extrinsic(R_t, params, X_3d, x_2d): # important
        return loss_function(params, R_t, X_3d, x_2d)

    result = least_squares(loss_function_extrinsic, R_t, args=(params, X_3d, x_2d), method='trf')
    
    optimized_params = result.x
    # R = optimized_params[:9].reshape(3,3)
    # t = optimized_params[9:]
    
    # extrinsic_optimized = (R, t.reshape(-1, 1))
    
    return optimized_params

In [28]:
def joint_optimization(K1_params, K2_params, R_t_1, R_t_2, triangulated_point, X1_homogeneous=X1_homogeneous, X2_homogeneous=X2_homogeneous):
    # Optimize extrinsic parameters
    R_t_1_opt = optimize_extrinsic_parameters(K1_params, R_t_1, triangulated_point, points2d_1)
    print("\n-----------------------------------------------------------------------------")
    R_t_2_opt = optimize_extrinsic_parameters(K2_params, R_t_2, triangulated_point, points2d_2)

    print("Optimized R_t_1:")
    print(R_t_1_opt)
    print("\nOptimized R_t_2:")
    print(R_t_2_opt)

    # Extract R and t from optimized parameters
    R_1_opt, t_1_opt = extract_R_t(R_t_1_opt)
    R_2_opt, t_2_opt = extract_R_t(R_t_2_opt)

    # Optimize intrinsic parameters
    K1_optimized_params = optimize_intrinsic_parameters(K1_params, R_t_1_opt, triangulated_point, points2d_1)
    print("\n-----------------------------------------------------------------------------")
    K2_optimized_params = optimize_intrinsic_parameters(K2_params, R_t_2_opt, triangulated_point, points2d_2)

    print("Optimized K1:")
    print(K1_optimized_params)
    print("\nOptimized K2:")
    print(K2_optimized_params)

    # Reconstruct final K matrices
    K1_final = construct_K_matrix(K1_optimized_params)
    K2_final = construct_K_matrix(K2_optimized_params)

    # Final triangulation
    P1_final = K1_final @ np.hstack((R_1_opt, t_1_opt.reshape(-1, 1)))
    P2_final = K2_final @ np.hstack((R_2_opt, t_2_opt.reshape(-1, 1)))
    X_3D_final = triangulate_points(X1_homogeneous, X2_homogeneous, P1_final, P2_final)

    return K1_optimized_params, K2_optimized_params, R_t_1_opt, R_t_2_opt, X_3D_final

Main Loop for joint optimization

In [ ]:
K1_final, K2_final, R_t_1_final, R_t_2_final, X_3D_final = joint_optimization(K1_optimized_params, K2_optimized_params, R_t_1, R_t_2, best_triangulation)

# Incursive
for i in range(2):
        K1_final, K2_final, R_t_1_final, R_t_2_final, X_3D_final= joint_optimization(K1_final, K2_final, R_t_1_final, R_t_2_final, X_3D_final)

print("\n---------------------------------------------------------------------")
print("final")
print("---------------------------------------------------------------------\n")
print(f"K1_final: \n{K1_final}\n")
print(f"K2_final: \n{K2_final}\n")
print(f"R_t_1_final: \n{R_t_1_final}\n")
print(f"R_t_2_final: \n{R_t_2_final}\n")
print(f"X_3D_final: \n{X_3D_final}\n")

Plotting the points after optimization.

In [ ]:
# For Camera 1
x_2d_cam1 = project_points(K1_final, R_t_1_final, X_3D_final)

print("x_2d_cam1:\n",x_2d_cam1)

normalized_cam1, normalized_2d_cam1 = normalize_points(x_2d_cam1)

print("\nnormalized\n", normalized_cam1, "\nnormalized_2d\n", normalized_2d_cam1)

plot_points(normalized_2d_cam1, "with optimized K1"), plot_points(points2d_1, "ground truth")

# For camera 2
x_2d_cam2 = project_points(K2_final, R_t_2_final, X_3D_final)

print("x_2d_cam2:\n",x_2d_cam2)

normalized_cam2, normalized_2d_cam2 = normalize_points(x_2d_cam2)

print("\nnormalized\n", normalized_cam2, "\nnormalized_2d\n", normalized_2d_cam2)

plot_points(normalized_2d_cam2, "with optimized K2"), plot_points(points2d_2, "ground truth")